In [2]:
#importing libraries
import pandas as pd
import pickle
from pandas import DataFrame, Series
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pybaseball as pyb
from pybaseball import statcast

In [3]:
#importing MLB Pitch Data
df = pd.read_csv("/Users/steph/OneDrive/Desktop/Python/PitchData2.csv",low_memory=False)
dft = pd.read_csv("/Users/steph/OneDrive/Desktop/Python/FastballData.csv",low_memory=False)

In [4]:
#Replace all the Description values of hit_into_play values with the Events values
df['Description'] = np.where(df['Description'] == 'hit_into_play', df['Events'], df['Description'])

In [5]:
#Create Differential Break Column and Side Release Column
df.insert(6, "Differential Break", df["Vertical Break"] - df["Horizontal Break"].abs(), True)
df.insert(9, "ABS Side Release", df["Side Release"].abs(), True)

In [6]:
df.head(5)

,Player Name,Pitch Type,Velocity,Spin Rate,Vertical Break,Horizontal Break,Differential Break,Release Height,Side Release,ABS Side Release,Extension,Side Location,Height Location,Events,Description,Swinging Strike
0,"Romano, Jordan",SL,87.4,2367.0,-2.40,6.36,-8.76,5.93,-2.22,2.22,7.3,0.74,2.15,strikeout,called_strike,0
1,"Romano, Jordan",SL,87.3,2203.0,-3.96,6.48,-10.44,5.62,-2.59,2.59,7.4,0.63,2.19,NaN,swinging_strike,1
2,"Romano, Jordan",FF,99.3,2344.0,16.32,-2.88,13.44,5.92,-2.16,2.16,7.5,-2.02,3.79,NaN,ball,0
3,"Romano, Jordan",FF,97.4,2407.0,16.32,-4.20,12.12,5.84,-2.37,2.37,7.4,-0.97,3.37,NaN,foul,0
4,"Romano, Jordan",SL,86.7,2258.0,-3.12,6.96,-10.08,5.54,-2.71,2.71,7.6,-0.07,1.86,grounded_into_double_play,grounded_into_double_play,0


In [7]:
#Create a new dataframe with only the columns we want to use and rename the columns
df = df[["Player Name","Velocity","Pitch Type","Spin Rate","Vertical Break","Horizontal Break","Release Height","ABS Side Release","Extension","Description"]]
df.rename(columns = {'ABS Side Release':'Side Release'}, inplace = True)

In [8]:
#Search for all the values in the Description column that are not in the Events column
df['Description'].value_counts()

Description
ball                         1926562
foul                         1003303
called_strike                 959351
field_out                     595156
swinging_strike               588925
single                        215024
blocked_ball                  132890
double                         66958
foul_tip                       50954
home_run                       47045
swinging_strike_blocked        43098
force_out                      30277
grounded_into_double_play      28746
hit_by_pitch                   15615
foul_bunt                      13945
field_error                    10725
sac_fly                         9590
intent_ball                     6756
triple                          6362
sac_bunt                        6335
double_play                     3456
fielders_choice                 3032
missed_bunt                     2976
fielders_choice_out             2463
pitchout                         793
bunt_foul_tip                    274
sac_fly_double_play       

In [9]:
#Set Run Value for each Description
home_run = 1.374328827219,
triple = 1.05755624961515,
double = 0.766083122898271,
single = 0.467292970729251
ball = 0.0636883289483747,
hit_by_pitch = 0.0636883289483747,
blocked_ball = 0.0636883289483747,
foul = -0.0380502742575014,
foul_tip = -0.0380502742575014,
bunt_foul = -0.0380502742575014,
bunt_foul_tip = -0.0380502742575014,
called_strike = -0.065092516089806,
swinging_strike = -0.118124935770601,
swinging_strike_blocked = -0.118124935770601,
force_out = -0.1955687665555,
grounded_into_double_play = -0.1955687665555,
fielders_choice_out = -0.1955687665555,
fielders_choice = -0.1955687665555,
field_out = -0.1955687665555,
double_play = -0.1955687665555,
Sac_fly = -0.236889645519856,
field_error = -0.236889645519856,
catcher_interf = -0.789788814378052
sac_fly_double_play = -0.789788814378052
triple_play = -0.789788814378052

In [10]:
#create A new column with the Run Value for each Description
df["Run Values"] = df["Description"].map({'home_run': 1.374328827219, 'triple': 1.05755624961515, 'double': 0.766083122898271, 'single': 0.467292970729251, 'ball': 0.0636883289483747, 'hit_by_pitch': 0.0636883289483747, 'blocked_ball': 0.0636883289483747, 'foul': -0.0380502742575014, 'foul_tip': -0.0380502742575014, 'bunt_foul': -0.0380502742575014, 'bunt_foul_tip': -0.0380502742575014, 'called_strike': -0.065092516089806, 'swinging_strike': -0.118124935770601, 'swinging_strike_blocked': -0.118124935770601, 'force_out': -0.1955687665555, 'grounded_into_double_play': -0.1955687665555, 'fielders_choice_out': -0.1955687665555, 'fielders_choice': -0.1955687665555, 'field_out': -0.1955687665555, 'double_play': -0.1955687665555, 'Sac_fly': -0.236889645519856, 'field_error': -0.236889645519856, 'catcher_interf': -0.789788814378052, 'sac_fly_double_play': -0.789788814378052, 'triple_play': -0.789788814378052})


In [11]:
#Create a new dataframe with only the columns we want to use and rename the columns
df = df[["Player Name","Velocity","Pitch Type","Spin Rate","Vertical Break","Horizontal Break","Description","Run Values"]]


In [12]:
df.head(5)

,Player Name,Velocity,Pitch Type,Spin Rate,Vertical Break,Horizontal Break,Description,Run Values
0,"Romano, Jordan",87.4,SL,2367.0,-2.40,6.36,called_strike,-0.065093
1,"Romano, Jordan",87.3,SL,2203.0,-3.96,6.48,swinging_strike,-0.118125
2,"Romano, Jordan",99.3,FF,2344.0,16.32,-2.88,ball,0.063688
3,"Romano, Jordan",97.4,FF,2407.0,16.32,-4.20,foul,-0.038050
4,"Romano, Jordan",86.7,SL,2258.0,-3.12,6.96,grounded_into_double_play,-0.195569


In [14]:
#convert dataframe to csv
df.to_csv('MLBRunValues.csv', index=False)

In [41]:
df_2 = pd.read_csv("C:/Users/steph/OneDrive/Documents/WBC_Pitching.csv")
df_2 = df_2[["Player Name","Velocity","Pitch Type","Spin Rate","Vertical Break","Horizontal Break","Description","Run Values"]]
df_2.head()

,Player Name,Velocity,Pitch Type,Spin Rate,Vertical Break,Horizontal Break,Description,Run Values
0,Young Pyo Ko,87.0,SI,1893.0,39,19.0,field_out,-0.195569
1,Fuyang Zhao,84.7,CH,1866.0,27,18.0,single,0.467293
2,Chih-Wei Hu,90.6,FF,2364.0,18,6.0,ball,0.063688
3,Kyle Glogoski,89.5,SI,2273.0,18,15.0,ball,0.063688
4,Shohei Ohtani,97.6,FF,2310.0,14,7.0,ball,0.063688


In [42]:
pd.concat([df, df_2]).to_csv('MLBRunValues.csv', index=False)

In [43]:
df_3 = pd.read_csv("C:/Users/steph/OneDrive/Desktop/Python/MLBRunValues.csv")
df_3.head()

,Player Name,Velocity,Pitch Type,Spin Rate,Vertical Break,Horizontal Break,Description,Run Values
0,"Romano, Jordan",87.4,SL,2367.0,-2.40,6.36,called_strike,-0.065093
1,"Romano, Jordan",87.3,SL,2203.0,-3.96,6.48,swinging_strike,-0.118125
2,"Romano, Jordan",99.3,FF,2344.0,16.32,-2.88,ball,0.063688
3,"Romano, Jordan",97.4,FF,2407.0,16.32,-4.20,foul,-0.038050
4,"Romano, Jordan",86.7,SL,2258.0,-3.12,6.96,grounded_into_double_play,-0.195569


In [36]:
df_3 = df_3[["Player Name","Velocity","Pitch Type","Spin Rate","Vertical Break","Horizontal Break","Description","Run Values"]]

In [44]:
print('Young Pyo Ko' in df_3['Player Name'].unique())

True


In [38]:
df_3.to_csv('MLBRunValues.csv', index=False)